In [1]:
using LinearAlgebra, ITensors
DEV = true
if DEV    # use local package
    include("../src/EDKit.jl")
    using .EDKit
else      # use EDKit in the Pkg system
    using EDKit
end

# Matrix Product States

The following function transform a many-body wave function to and from an MPS.

In [35]:
function vec2mps(
    vec::AbstractVector; 
    base::Integer=2, 
    L::Integer=round(Int, log(base, length(vec))), 
    cutoff::Real=1e-8, 
    maxdim::Integer=base^div(L, 2)
)
    sites = [Index(base, "Site, n=$i") for i in 1:L]
    # reverse sites because Julia's column-major convention.
    T = ITensor(reshape(vec, fill(base, L)...), reverse(sites))
    MPS(T, sites; cutoff=cutoff, maxdim=maxdim)
end

function mps2vec(psi::MPS)
    s = siteinds(psi)
    T = prod(t for t in psi)
    A = array(T, reverse(s))
    A[:]
end

function element(psi::MPS, el::Union{<:AbstractVector, <:Tuple})
    s = siteinds(psi)
    V = ITensor(1.)
    for j in eachindex(psi)
        V *= psi[j] * state(s[j], el[j])
    end
    scalar(V)
end
element(psi::MPS, el::Integer...) = element(psi::MPS, el)

element (generic function with 2 methods)

In [67]:
L = 4
base = 3
vec = rand(ComplexF64, base^L) |> normalize
psi = vec2mps(vec, base=base)

ind = rand(1:base, L)
I = index(ind .- 1, base=base)
err1 = abs(vec[I] - element(psi, ind))
@show err1

vec2 = mps2vec(psi)
err2 = norm(vec-vec2)
@show err2;

err1 = 1.1739862079479962e-16
err2 = 9.232648246240546e-16
